In [103]:
import pandas as pd
from scipy import stats
import datetime
from sklearn.metrics import mean_squared_error
import rpy2
import math
import seaborn as sns
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [193]:
df = pd.read_csv("ecs_series_1_uid_encoded(1).csv")

In [194]:
datetime_object = datetime.datetime.strptime("9/27/18 16:00", "%m/%d/%y %H:%M")
df['Date'] = df["ds"].apply(lambda x:datetime.datetime.strptime(x, "%m/%d/%y %H:%M"))
df = df.drop(columns = ['ds','region_no_factor','iz_no_factor','product_factor','instance_type_name_factor','instance_type_family_factor'])

In [195]:
df = df.groupby(by=['uid','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'uid',values = 'vcpu_net_delead')

In [67]:
df_2000 = pd.read_csv("test.csv")
#df_2000 = df.sample(2000)

## Theta method

In [58]:
from rpy2.robjects.packages import importr
# import R's "base" package
# import rpy2's package module
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
# R package names
packnames = ['forecast']
# R vector of strings
from rpy2.robjects.vectors import StrVector
utils.install_packages(StrVector('forecast'))
base = importr('forecast')


In [68]:
# Theta just skips nan
df_2000 = df_2000.applymap(lambda x:float(0.0) if (np.isnan(x)) else x)

### We take test as last 10 weeks (70 days)

In [80]:
df_2000_train= df_2000.iloc[:,0:len(df_2000.columns)-70]
df_2000_true= df_2000.iloc[:,len(df_2000_train.columns):len(df_2000.columns)]
df_2000_true.insert(0,'uid',df_2000['uid'])

df_2000_train = df_2000_train.set_index('uid')
df_2000_true=df_2000_true.set_index('uid')

In [98]:
df_prediction = pd.DataFrame(columns = df_2000_true.columns)
for index, row in df_2000_train.iterrows():
    res = robjects.FloatVector(row.drop(columns = ['uid']))
    r_prediction = base.thetaf(res,70)[1]
    df_prediction.loc[index] = r_prediction

In [120]:
math.sqrt(mean_squared_error(df_2000_true,df_prediction))

0.06484291101699256

In [121]:
def smape_(y_true, y_pred, mean = True):
    #smape = 2 * np.sum(np.abs(y_pred-y_true) / np.abs(y_true + y_pred), axis = 0)*100   
    smape = 2 * np.sum(np.abs(y_pred-y_true) / np.abs(y_true + y_pred), axis = 0)*100/len(y_pred)
    if mean == True:
        smape = smape.mean()
    return smape

In [122]:
smape_(df_2000_true,df_prediction)

164.3439840559633

In [125]:
df_monetary = pd.DataFrame(columns = df_2000_true.columns)
for index, row in df_2000_true.iterrows():
    arr = []
    arr = list(row)
    rec_arr = []
    mon_arr = []
    freq_arr = []
    start_date = row.index[0]
    last_purchase = np.nan
    first_purchase = 0 #flag
    total_purchases = 0
    current_servers = 0
    total_days_of_purchases = 0 
    for j,entry in enumerate(arr):
        if (np.isnan(entry)):
            total_days_of_purchases+=current_servers
        else:
            current_servers+=entry
            total_days_of_purchases+=current_servers
        mon_arr.append(total_days_of_purchases/(j+1))
    df_monetary.loc[index] = mon_arr

In [132]:
df_monetary

,2018-09-17 16:00:00,2018-09-18 16:00:00,2018-09-19 16:00:00,2018-09-20 16:00:00,2018-09-21 16:00:00,2018-09-22 16:00:00,2018-09-23 16:00:00,2018-09-24 16:00:00,2018-09-25 16:00:00,2018-09-26 16:00:00,...,2018-11-16 16:00:00,2018-11-17 16:00:00,2018-11-18 16:00:00,2018-11-19 16:00:00,2018-11-20 16:00:00,2018-11-21 16:00:00,2018-11-22 16:00:00,2018-11-23 16:00:00,2018-11-24 16:00:00,2018-11-25 16:00:00
117237,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.051366,-0.051613,-0.051852,-0.052083,-0.052308,-0.052525,-0.052736,-0.052941,-0.053140,-0.053333
7669,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
117808,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.048087,-0.048387,-0.048677,-0.048958,-0.049231,-0.049495,-0.049751,-0.050000,-0.050242,-0.050476
34612,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.010458,-0.018824,...,-0.081774,-0.081973,-0.082166,-0.082353,-0.082534,-0.082709,-0.082880,-0.083045,-0.083205,-0.083361
74579,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
143086,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000505,0.000995,0.001471,0.001932,0.002381
4170,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
77909,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
97054,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
79194,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [126]:
df_prediction_monetary = pd.DataFrame(columns = df_monetary.columns)
for index, row in df_2000_train.iterrows():
    res = robjects.FloatVector(row.drop(columns = ['uid']))
    r_prediction = base.thetaf(res,70)[1]
    df_prediction_monetary.loc[index] = r_prediction

In [129]:
math.sqrt(mean_squared_error(df_monetary,df_prediction_monetary))

0.19443078647299353

In [130]:
smape_(df_monetary,df_prediction_monetary)

181.81741818270123

In [ ]:
df_monetary

In [131]:
df_prediction_monetary

,2018-09-17 16:00:00,2018-09-18 16:00:00,2018-09-19 16:00:00,2018-09-20 16:00:00,2018-09-21 16:00:00,2018-09-22 16:00:00,2018-09-23 16:00:00,2018-09-24 16:00:00,2018-09-25 16:00:00,2018-09-26 16:00:00,...,2018-11-16 16:00:00,2018-11-17 16:00:00,2018-11-18 16:00:00,2018-11-19 16:00:00,2018-11-20 16:00:00,2018-11-21 16:00:00,2018-11-22 16:00:00,2018-11-23 16:00:00,2018-11-24 16:00:00,2018-11-25 16:00:00
117237,0.000193,0.000193,0.000193,0.000193,0.000194,0.000194,0.000194,0.000194,0.000195,0.000195,...,0.000207,0.000207,0.000207,0.000207,0.000208,0.000208,0.000208,0.000208,0.000208,0.000209
7669,-0.000009,-0.000009,-0.000009,-0.000009,-0.000009,-0.000009,-0.000009,-0.000009,-0.000009,-0.000009,...,-0.000009,-0.000009,-0.000009,-0.000009,-0.000010,-0.000010,-0.000010,-0.000010,-0.000010,-0.000010
117808,0.000195,0.000195,0.000195,0.000195,0.000195,0.000196,0.000196,0.000196,0.000196,0.000197,...,0.000209,0.000209,0.000209,0.000209,0.000209,0.000210,0.000210,0.000210,0.000210,0.000211
34612,0.000222,0.000222,0.000223,0.000223,0.000223,0.000223,0.000224,0.000224,0.000224,0.000224,...,0.000237,0.000237,0.000237,0.000237,0.000238,0.000238,0.000238,0.000238,0.000239,0.000239
74579,0.000123,0.000123,0.000123,0.000123,0.000123,0.000123,0.000123,0.000124,0.000124,0.000124,...,0.000130,0.000130,0.000130,0.000130,0.000130,0.000130,0.000130,0.000130,0.000131,0.000131
143086,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4170,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,...,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073,0.000074,0.000074,0.000074
77909,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000126,0.000126,0.000126,...,0.000132,0.000132,0.000132,0.000132,0.000132,0.000132,0.000133,0.000133,0.000133,0.000133
97054,0.000156,0.000156,0.000156,0.000156,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,...,0.000166,0.000166,0.000166,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000168
79194,-0.000173,-0.000173,-0.000173,-0.000173,-0.000174,-0.000174,-0.000174,-0.000174,-0.000174,-0.000175,...,-0.000185,-0.000185,-0.000185,-0.000185,-0.000185,-0.000186,-0.000186,-0.000186,-0.000186,-0.000186
